In [0]:
"""
# 04_gold_customer_growth
Descrição: Gera visão agregada de aquisição de clientes por mês (Safra/Cohorts).
Fonte: Silver Customers
"""
from pyspark.sql.functions import *

In [0]:
# 1. Configuração
source_table = "main.default.silver_customers"
target_table_name = "main.default.gold_customer_growth"
volume_path = "/Volumes/main/default/lakehouse/gold_customer_growth" # Opcional, pois é gerenciada

In [0]:
# 2. Leitura da Silver
# AQUI ESTÁ O SEGREDO:
# Para contar "Novos Clientes", não nos importamos se ele mudou de endereço (SCD2).
# Nos importamos com quem ele é (ID) e quando chegou (signup_date).
# Por isso, fazemos um distinct para pegar cada cliente apenas uma vez.

df_silver = spark.read.table(source_table)

df_unique_customers = (df_silver
    .select("customer_id", "signup_date")
    .distinct() # Garante que clientes com múltiplas linhas (histórico) contem apenas 1 vez
)

In [0]:
# 3. Tratamento e Agregação
df_growth = (df_unique_customers
    # Cria o "truncamento" da data para o primeiro dia do mês (Ex: 2025-11-15 -> 2025-11-01)
    .withColumn("acquisition_month", date_trunc("month", col("signup_date")))
    .groupBy("acquisition_month")
    .agg(
        count("customer_id").alias("new_customers_count")
    )
    .orderBy("acquisition_month")
)

In [0]:
# 4. Escrita (Tabela Gerenciada)
(df_growth.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(target_table_name)
)

print(f"✅ Tabela de Crescimento '{target_table_name}' gerada com sucesso!")
display(df_growth)